In [1]:
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11.0.11"
os.environ["SPARK_HOME"] = f"C:\spark\spark-3.1.2-bin-hadoop3.2"
import findspark
import pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import config
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable

In [2]:
rds_string = config.rds_string
engine = create_engine(f'postgresql://{rds_string}')
conn = engine.connect()
metadata = MetaData(engine)

In [3]:
def sql_to_spark(table):
    from pyspark.sql import Row
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    query_list = []
    for i in query:
        q_len = len(i)
        temp_dict = {}
        for j in range(q_len):
            key = steve[j]
            value = i[j]
            if value == None:
                value = float(0)
            temp_dict[key] = value
            if j == (q_len - 1):
                query_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in query_list)
    return df

def query_maker(table):
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    return (query, steve)

def spark_maker(query,country):
    c_list = []
    for i in query:
        q_len = len(i)
        if i[0] == country:
            temp_dict = {}
            for j in range(q_len):
                key = steve[j]
                value = i[j]
                if value == None:
                    value = float(0)
                temp_dict[key] = value
                if j == (q_len - 1):
                    c_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in c_list) 
    return df

def sql_spark_trade(table, country):
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).filter(table_var.c.country_or_area == country).all()
    steve = table_var.metadata.tables[table].columns.keys()
    c_list = []
    for i in query:
        q_len = len(i)
        if i[0] == country:
            temp_dict = {}
            for j in range(q_len):
                key = steve[j]
                value = i[j]
                if value == None:
                    value = float(0)
                temp_dict[key] = value
                if j == (q_len - 1):
                    c_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in c_list)
    return df

def melt(
        df: DataFrame, 
        id_vars: Iterable[str], value_vars: Iterable[str], 
        var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name)) 
        for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

In [4]:
from sqlalchemy import MetaData, Table
metadata = MetaData(engine)
global_trade = Table("global_trade", metadata, autoload=True, autoload_with=engine)
trade_schema = global_trade.metadata.tables["global_trade"].columns.keys()
trade_schema

In [14]:
spark_gdp=sql_to_spark('gdp_data')
spark_india=sql_to_spark("india_export_data")
spark_emissions=sql_to_spark('global_emissions')

In [27]:
india_trade_spark = sql_spark_trade("global_trade","India")
india_trade_spark_y = india_trade_spark.withColumn("year",india_trade_spark.year.cast('int'))
spark_gdp_2 = spark_gdp_2.withColumnRenamed("variable","year")


In [28]:
india_export_df = india_trade_spark_y.filter(india_trade_spark_y['flow'] == "Export")
india_import_df = india_trade_spark_y.filter(india_trade_spark_y['flow'] == "Import")

In [29]:
india_trade_spark_y.orderBy(india_trade_spark_y.year.asc()).show(1)

In [18]:
gdp_col = spark_gdp.columns
gdp_col2 = gdp_col[2:]
gdp_static_cols = gdp_col[:2]
spark_gdp_2 = melt(spark_gdp, id_vars=gdp_static_cols, value_vars=gdp_col2)
spark_gdp_2 = spark_gdp_2.withColumn("variable",spark_gdp_2.variable.cast('int'))
spark_gdp_2 = spark_gdp_2.withColumnRenamed("variable","year")
spark_gdp_2 = spark_gdp_2.withColumnRenamed("value","GDP")
spark_gdp_2 = spark_gdp_2.select(['Country ','year','GDP'])

In [21]:
india_gdp_spark = spark_gdp_2.filter(spark_gdp_2["Country "] == "India")

In [23]:
spark_emissions = spark_emissions.withColumnRenamed("Entity","Country")

In [69]:
india_emissions = spark_emissions.filter(spark_emissions["Country"] == "India")

In [87]:
india_export_df.show(5)

+---------------+----+---------+--------------------+------+---------+-----------+-------------------+-----------+--------------------+
|country_or_area|year|comm_code|           commodity|  flow|trade_usd|  weight_kg|      quantity_name|   quantity|            category|
+---------------+----+---------+--------------------+------+---------+-----------+-------------------+-----------+--------------------+
|          India|2016|   210110|Coffee extracts, ...|Export|276962932|4.0752197E7|Weight in kilograms|4.0752197E7|21_miscellaneous_...|
|          India|2016|   210120|Tea and mate extr...|Export| 43467004|  6351410.0|Weight in kilograms|  6351410.0|21_miscellaneous_...|
|          India|2016|   210130|Chicory & other c...|Export| 17190546|1.8826556E7|Weight in kilograms|1.8826556E7|21_miscellaneous_...|
|          India|2016|   210210|      Yeasts, active|Export|   331233|   224155.0|Weight in kilograms|   224155.0|21_miscellaneous_...|
|          India|2016|   210220|Yeasts, inactive

In [89]:
india_import_df.show(5)

+---------------+----+---------+--------------------+------+---------+---------+-------------------+---------+--------------------+
|country_or_area|year|comm_code|           commodity|  flow|trade_usd|weight_kg|      quantity_name| quantity|            category|
+---------------+----+---------+--------------------+------+---------+---------+-------------------+---------+--------------------+
|          India|2016|   210110|Coffee extracts, ...|Import| 13371521|1881816.0|Weight in kilograms|1881816.0|21_miscellaneous_...|
|          India|2016|   210120|Tea and mate extr...|Import|  1988772| 105363.0|Weight in kilograms| 105363.0|21_miscellaneous_...|
|          India|2016|   210130|Chicory & other c...|Import|    17363|   1266.0|Weight in kilograms|   1266.0|21_miscellaneous_...|
|          India|2016|   210210|      Yeasts, active|Import| 15532547|4839851.0|Weight in kilograms|4839851.0|21_miscellaneous_...|
|          India|2016|   210220|Yeasts, inactive,...|Import|  1317256|183324

In [85]:
spark_india.show(5)

+------+--------------------+-----+---------------+----+
|HSCode|           Commodity|value|        country|year|
+------+--------------------+-----+---------------+----+
|     2|MEAT AND EDIBLE M...| 0.18|AFGHANISTAN TIS|2018|
|     3|FISH AND CRUSTACE...|  0.0|AFGHANISTAN TIS|2018|
|     4|DAIRY PRODUCE; BI...|12.48|AFGHANISTAN TIS|2018|
|     6|LIVE TREES AND OT...|  0.0|AFGHANISTAN TIS|2018|
|     7|EDIBLE VEGETABLES...| 1.89|AFGHANISTAN TIS|2018|
+------+--------------------+-----+---------------+----+
only showing top 5 rows



In [86]:
india_gdp_spark.show(5)

+--------+----+-----------+
|Country |year|        GDP|
+--------+----+-----------+
|   India|1990|1236.671208|
|   India|1991|1265.917481|
|   India|1992|1338.664595|
|   India|1993|1407.473802|
|   India|1994|1503.745417|
+--------+----+-----------+
only showing top 5 rows



In [84]:
india_emissions.show(5)

+-------+----+---------------------------+
|Country|Year|annual_co2_emissions_tonnes|
+-------+----+---------------------------+
|  India|1751|                        0.0|
|  India|1752|                        0.0|
|  India|1753|                        0.0|
|  India|1754|                        0.0|
|  India|1755|                        0.0|
+-------+----+---------------------------+
only showing top 5 rows

